# Probability Modelling in fmdtools
Here we will demonstrate the fmdtool toolkit's abilities to find a set of scenarios that represent the effects of a given mode.

In [1]:
#First, import the fault propogation library as well as the model
#since the package is in a parallel location to examples...
import fmdtools.sim.propagate as propagate
import fmdtools.analyze as an

In [2]:
import fmdtools
help(fmdtools)

Help on package fmdtools:

NAME
    fmdtools

PACKAGE CONTENTS
    analyze (package)
    define (package)
    sim (package)

DATA
    __all__ = ['analyze', 'sim', 'define']

FILE
    c:\users\dhulse\documents\github\fmdtools\fmdtools\__init__.py




In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import quadpy
from ex_pump import *
from IPython.display import HTML
mdl = Pump()

TypeError: __init__() got an unexpected keyword argument 'params'

## Theory and Goals

The purpose of this document is to show the relative performance of various numerical integration techniques for resilience quantification.


### Some approaches:

- monte carlo with and without variance reduction

- quadrature (from quadpy)

- sample likeliest phase (`maxlike` option)


## Model Description
 pump: defined in ex_pump. Has the following functions:
 - import EE
 - import Water
 - import Signal
 - move Water
 - export Water
 
and flows:
- EE (to power the pump)
- Water_in
- Water_out
- Signal input (on/off)

In [4]:
an.graph.show(mdl)
plt.tight_layout(pad=-1)

In [5]:
endresults, mdlhist=propagate.one_fault(mdl, 'ExportWater', 'block', time=10, staged=True)
#rp.show_graph(resgraph)

The following shows the states of the system over time with and without a fault:

In [6]:
fig = an.plot.mdlhists(mdlhist, cols=3)

In [7]:
fig, ax = an.plot.mdlhistvals(mdlhist, 'blockage', time=10,fxnflowvals={'Wat_2':['flowrate', 'pressure'],'EE_1':'current'}, cols=2, returnfig=True, timelabel="time (min)")
#plt.rcParams.update({'font.size': 14})
fig.axes[0].set_ylabel("Amps")
fig.axes[1].set_ylabel("$m^3/s$")
fig.axes[2].set_ylabel("kPa")
fig.tight_layout(pad=0.3)
plt.subplots_adjust(top=0.93)

In [8]:
fig, ax = an.plot.mdlhists(mdlhist,fxnflowvals={'Wat_2':['flowrate', 'pressure'],'EE_1':'current'}, cols=2,xlabel="time (min)",\
                           ylabels={('flows', "EE_1",'current'):'Amps', ('flows', 'Wat_2', 'flowrate'):"$m^3/s$", ('flows', 'Wat_2', 'pressure'):"kPa"},\
                          time_slice=[10], time_slice_label='fault time', title="Pump Response to Blockage", h_padding=0.3)


In [9]:
fig.savefig("behaviorovertime.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [10]:
fig.savefig("behaviorovertime.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [11]:
deghist_fault = mdlhist.get_degraded_hist(*mdl.fxns, *mdl.flows)
fault_fault = mdlhist.get_faulty_hist(*mdl.fxns, *mdl.flows)

fig, axis = an.graph.result_from(mdl, deghist_fault, 45, scale=1.0, gtype='fxngraph')
axis.set_title("Pump System with Blockage Fault")

In [12]:
fig.savefig("modelgraph.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

Each of these flows also have different different costs associated with degraded performance:
 - repair cost
 - lost water cost
 - ee use cost
 These costs also have different forms. 

## Experiments

The goal of these experiments is to compare the error and efficiency of quadrature approaches at approximating the expected resilience over the simulation interval. 

### Introduce Quadratures
The quadratures used in this experiment are shown below. While quadpy has a few other quadratures that can be used, these were chosen becuase they take a single argument for the number of points used.

In [13]:
quadratures = {}

In [14]:
quadratures['gauss-legendre'] = quadpy.c1.gauss_legendre
quadratures['gauss-legendre'](3).show()
quadratures['gauss-legendre'](3).points


In [15]:
quadratures['fejer 1'] = quadpy.c1.fejer_1 #argument is # of points
quadratures['fejer 1'](5).show()
quadratures['fejer 1'](5).points

In [16]:
quadratures['fejer 2'] = quadpy.c1.fejer_2 #argument is # of points
quadratures['fejer 2'](5).show()
quadratures['fejer 2'](5).points

In [17]:
quadratures['gauss lobatto'] = quadpy.c1.gauss_lobatto #argument is # of points
quadratures['gauss lobatto'](5).show() 
(quadratures['gauss lobatto'](5).points)

In [18]:
#argument is number of pts, (optional) bias left, bias right (maybe try flipping?)
#quadratures['gauss radau'] = quadpy.line_segment.gauss_radau
#quadratures['gauss radau'](3).show()

In [19]:
#argument is n which makes the number of pts n+1
#quadratures['newton cotes closed'] = quadpy.line_segment.newton_cotes_closed 
#quadratures['newton cotes closed'](2).show()

In [20]:
#quadratures['newton cotes open'] = quadpy.line_segment.newton_cotes_open #argument num of pts
#quadratures['newton cotes open'](3).show() 

Some other possible quadratures are shown below. 

In [21]:
#Gauss-kronrod
quadpy.c1.gauss_kronrod(5).show() #argument is n which makes the number off pts 2*n+1
#Gauss-jacobi
quadpy.c1.gauss_jacobi(5, 1,2).show() #argument is # of points, left weighting, right weighting > -1
#Gauss-patterson
quadpy.c1.gauss_patterson(3).show() #argument is the order n, num pts is [1, 3, 7, 15, 31, 63, 127, 255 or 511]

The code below calculates the error of each approach over the number of points used. For the monte-carlo approaches, the results are averaged over 10 simulations and the standard deviation is provided for error bars. The computational cost is also determined by determined by taking the number of simulations needed for each (which is 1 per point (from simulating the nominal) and a fraction based on the injection time of the simulation (faults injected later take less time in simulation)).

### Experimental setup
The following function calculates the expected resilience of a model `mdl` (as well as a few other useful things) over the fault sampling approach defined in `approach`

In [22]:
def resilquant(approach, mdl):
    endclasses, mdlhists = propagate.approach(mdl, approach, showprogress=False)
    
    fmea = an.tabulate.summfmea(endclasses, approach)
    fmea2 = an.tabulate.phasefmea(endclasses, approach)
    util=sum(fmea['expected cost'])

    return util, fmea, fmea2

This experiment will test how well the quadratures approximate the expected costs of resilience by sampling different points in interval. This is done for the pump model introduced above using the blockage fault.

Below each of the cost types are shown:
 - repair costs (which is a step function due to the conditional fault that occurs due to delays)
 - water costs (which is linear due to the water lost at each timestep)
 - EE costs (which is constant and then exponentially decreasing due to the interval of adverse high current draw)
 - all costs together
 
 where the delay for degradation is 20 time-steps

In [23]:
costs = [{'repair'}, {'water'}, {'ee'}, {'repair', 'water', 'ee'}]
costtitle={'repair':'Repair', 'water':' Decreased Flow', 'ee': 'Increased Current'}
costfig = plt.figure(figsize=(5,4))
costfig.add_subplot(2, 2, 4)
plt.rcParams.update({'font.size': 11})
n=1
for cost in costs:
    mdl = Pump(params={'cost':cost, 'delay':20})
    app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
    endclasses, mdlhists = propagate.approach(mdl, app_full, showprogress=False)
    ax = costfig.add_subplot(2, 2, n)
    an.plot.metricovertime(endclasses, app_full, metric='cost', metrictype='cost')
    if len(cost)>1:
        plt.title("Composite")
    else:
        plt.title(costtitle[list(cost)[0]])
    plt.ylim([0,30000])
    plt.xlim(0,55)
    if n%2<1:
        ax.set_yticklabels([])
        ax.set_ylabel("")
    else:
        ax.set_ylabel("Cost (USD)")
    if n<3:
        ax.set_xticklabels([])
        ax.set_xlabel("")
    n+=1
costfig.suptitle("Cost of Faults Over Injection Times")
plt.subplots_adjust(left=0.2)

In [24]:
plt.savefig("costsovertime.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

### Comparing Quadrature Error

The functions below compare the result from a set of sampling approaches with the full integral given that quadrature can use a certain amount of points to approximate the function. The error being used is absolute error.

In [25]:
def calcerror(fullutil, util):    
    return abs((fullutil-util)/(fullutil+0.000001))

In [26]:
def calc_errors(mdl, quadratures, possible_pts, util_full):
    errs = dict.fromkeys(quadratures)
    num_samps = dict.fromkeys(quadratures)
    for quadname, quad in quadratures.items():
        errs[quadname] = []
        num_samps[quadname] = []
        if quadname=='likeliest':
            app=SampleApproach(mdl,faults=[('ExportWater' ,'block')],defaultsamp={'samp':'likeliest'})
            util, fmea, phasefmea = resilquant(app, mdl)
            errs[quadname] = errs[quadname] + [calcerror(util_full, util)]
            num_samps[quadname] = num_samps[quadname] + [len(app.times) + sum([(55-scen.time)/55 for scen in app.scenlist])]
        else:
            for pt in possible_pts:
                try:
                    if quadname == 'evenspacing': app=SampleApproach(mdl,faults=[('ExportWater' ,'block')],defaultsamp={'samp':'evenspacing','numpts':pt})
                    else:  
                        quadobj = quad(pt)
                        app=SampleApproach(mdl,faults=[('ExportWater' ,'block')],defaultsamp={'samp':'quadrature','quad':{'nodes':quadobj.points,'weights':quadobj.weights}})
                    util, fmea, phasefmea = resilquant(app, mdl)
                    errs[quadname] = errs[quadname] + [calcerror(util_full, util)]
                    num_samps[quadname] = num_samps[quadname] + [len(app.times) + sum([(55-scen.time)/55 for scen in app.scenlist])]
                except: 0
    return errs, num_samps

Paremeters for the experiment:

In [27]:
possible_pts = range(1,15) # points for the quadrature
#quadratures['evenspacing'] = 1   # adding evenly spaced sampling 
#quadratures['likeliest'] = 1     # adding likeliest phase sampling
delaytimes = [12, 23, 34]        # delay times to test

Calculating the error over each cost type, delay, and number of points

In [28]:
errovercosts=dict()
for cost in costs:
    errovercosts[str(cost)]=dict()
    for delay in delaytimes:
        n+=1
        mdl = Pump(params={'cost':cost, 'delay':delay})
        app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
        util_full, fmea_full, phasefmea_full = resilquant(app_full, mdl)
        errovercosts[str(cost),delay], num_samps = calc_errors(mdl, quadratures, possible_pts, util_full)

Plotting results:

In [29]:
figure = plt.figure(figsize=(11, 8))
plt.rcParams.update({'font.size': 12})
figure.add_subplot(len(costs), len(delaytimes), len(costs)*len(delaytimes))
n=0
markers = {"gauss-legendre":"o","fejer 1":"v", "fejer 2":"^", "gauss lobatto":"s"}
labs = {"gauss-legendre":"Gauss-Legendre","fejer 1":"Fejer 1", "fejer 2":"Fejer 2", "gauss lobatto":"Gauss-Lobatto"}
for cost in costs:
    for delay in delaytimes:
        n+=1
        ax = figure.add_subplot(len(costs), len(delaytimes), n)
        for quad in errovercosts[str(cost), delay]:
            ax.plot(num_samps[quad], errovercosts[str(cost), delay][quad], marker=markers[quad], label=labs[quad])#, label=labels[quad], marker=markers[quad])
        title = list(cost)
        if len(title)>1: title="Composite"
        else: title = costtitle[list(cost)[0]]
        ax.set_title(title+", "+str(delay))
        ax.set_yscale("symlog", linthresh=0.001)
        ax.set_ylim([0,0.5])
        ax.grid(True)
        if delay != min(delaytimes): ax.set_yticklabels([])
        if title != "Composite": ax.set_xticklabels([])
        handles, labels = ax.get_legend_handles_labels()
figure.suptitle("Error of Quadratures Over # of Points, Cost Types, and Delay Parameter")
figure.legend(handles, labels, loc="center right") #bbox_to_anchor=(1.1, 0.5))
figure.text(0.5, 0.04, 'Number of Full Model Runs', ha='center')
figure.text(0.04, 0.5, 'Error', va='center', rotation='vertical')

figure.tight_layout(pad=0.3)
plt.subplots_adjust(right=0.80, left=0.1, top=0.91, bottom=0.1)

In [30]:
plt.savefig("quad_errors.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [31]:
avg_cost=pd.DataFrame()
for cost in costs:
    for delay in delaytimes:
        ed=errovercosts[str(cost), delay]
        df_cost = pd.DataFrame.from_dict(dict([(k,pd.Series(v)) for k,v in ed.items()]))
        df_cost["gauss lobatto"]=df_cost["gauss lobatto"].shift(1)
        avg_cost[str(cost),delay]=df_cost[1:11].mean()
avg_cost = avg_cost.transpose()
avg_cost

In [32]:
averages = pd.DataFrame([avg_cost[0:3].mean(),avg_cost[3:6].mean() ,avg_cost[6:9].mean(), avg_cost[9:12].mean(), avg_cost.mean()])
averages["names"]=["repair", "water", "ee", "composite", "average"]
averages= averages[["names", "gauss-legendre", "fejer 1", "fejer 2", "gauss lobatto"]]
averages

In [33]:
averages.to_latex()

If individual plots are desired:

In [34]:
def plot_errors(errs, num_samps, showlegend=True):
    for quad in errs:
        plt.plot(num_samps[quad], errs[quad], label=quad, marker='o')
    if showlegend: plt.legend(loc='upper right')
    plt.xlabel("Number of Full Model Runs")
    plt.yscale("symlog", linthreshy=0.001)
    plt.grid('both')
    plt.ylabel("Percent Error")

In [35]:
for delay in delaytimes:
    for cost in costs:
        mdl = Pump(params={'cost':cost, 'delay':delay})
        app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
        util_full, fmea_full, phasefmea_full = resilquant(app_full, mdl)
        errs, num_samps = calc_errors(mdl, quadratures, possible_pts, util_full)
        plt.figure()
        plt.title("Error of Sampling Approaches (cost: "+str(cost)+" delay: "+str(delay))
        plot_errors(errs, num_samps)

What if the form of the integral was known beforehand? In that case one would be interested in knowing the points that best represente the space for future iteration. There are two options for that in this model: 'single-point' and 'piecewise'

In [36]:
mdl = Pump(params={'cost':{'water', 'repair', 'ee'}, 'delay':10})
app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
app_piecewise = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
app_bestpt = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
endclasses, mdlhists = propagate.approach(mdl, app_full, showprogress=False)
app_piecewise.prune_scenarios(endclasses, samptype='piecewise')
app_bestpt.prune_scenarios(endclasses, samptype='bestpt')

delaytimes = [i for i in range(0,45,1)]
piecewiseerr=[]
bestpterr=[]
for delay in delaytimes:
    mdl = Pump(params={'cost':{'water', 'repair', 'ee'}, 'delay':delay, 'units':'hrs'}) 
    util_full,  fmea_full, phasefmea_full = resilquant(app_full, mdl)
    util_piecewise,  fmea_piecewise, phasefmea_piecewise = resilquant(app_piecewise, mdl)
    util_bestpt,  fmea_bestpt, phasefmea_bestpt = resilquant(app_bestpt, mdl)
    piecewiseerr=piecewiseerr+[calcerror(util_full, util_piecewise)]
    bestpterr = bestpterr +[calcerror(util_full, util_bestpt)]


In [37]:
plt.figure()
plt.plot(delaytimes, piecewiseerr, label='piecewise')
plt.axvline(10, color='black')
plt.plot(delaytimes, bestpterr, label='bestpt')
plt.legend()

This may change over the delay time parameters chosen. The code below plots the value over a set of possible delay paremeters.

In [38]:
postplot = plt.figure(figsize=(5,4))
plt.rcParams.update({'font.size': 12})
postplot.add_subplot(2,2,4)
n=0
for initdelay in [9,18,27,36]:
    n+=1
    mdl = Pump(params={'cost':{'water', 'repair', 'ee'}, 'delay':initdelay})
    app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
    app_piecewise = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
    app_bestpt = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
    endclasses, mdlhists = propagate.approach(mdl, app_full, showprogress=False)
    app_piecewise.prune_scenarios(endclasses, samptype='piecewise')
    app_bestpt.prune_scenarios(endclasses, samptype='bestpt')

    delays = [i for i in range(0,46,1)]
    piecewiseerr=[]
    bestpterr=[]
    for delay in delays:
        mdl = Pump(params={'cost':{'water', 'repair', 'ee'}, 'delay':delay}) 
        util_full, fmea_full, phasefmea_full = resilquant(app_full, mdl)
        util_piecewise,  fmea_piecewise, phasefmea_piecewise = resilquant(app_piecewise, mdl)
        util_bestpt,  fmea_bestpt, phasefmea_bestpt = resilquant(app_bestpt, mdl)
        piecewiseerr=piecewiseerr+[calcerror(util_full, util_piecewise)]
        bestpterr = bestpterr +[calcerror(util_full, util_bestpt)]
    
    ax = postplot.add_subplot(2,2,n)
    plt.ylim([0,0.5])
    plt.plot(delays, piecewiseerr, label='piecewise', color='blue')
    plt.plot(delays, bestpterr, label='bestpt', linestyle='--', color='grey')
    handles, labels = ax.get_legend_handles_labels()
    plt.axvline(initdelay, color='black', label = 'init. delay')
    plt.title("Initial delay = "+str(initdelay))
    plt.grid()
    if n%2<1:
        ax.set_yticklabels([])
    else:
        ax.set_ylabel("Error")
    if n<3:
        ax.set_xticklabels([])
    else:
        ax.set_xlabel("Time (min)")
postplot.suptitle("A Posteriori Approach Robustness")
postplot.tight_layout()
postplot.subplots_adjust(top=0.88)
postplot.legend(handles, labels, bbox_to_anchor=(0.7, 0.4))

In [39]:
postplot.savefig("posthoc_err.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

This gives an overall look at how quadratures compare with other types of fault sampling approaches:

In [40]:
app_full = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'fullint'})
util_full, fmea_full, phasefmea_full = resilquant(app_full, mdl)

In [41]:
possible_pts=range(1,45)
samp_types = ['evenspacing', 'randtimes', 'symrandtimes', 'quadrature']
errs=dict.fromkeys(samp_types)
errbars=dict.fromkeys(samp_types)
errbar_samps = dict.fromkeys(samp_types)
num_samples = dict.fromkeys(samp_types)
ave_pts=20

util_full, fmea_full, phasefmea_full = resilquant(app_full, mdl)
for samp_type in samp_types:
    errs[samp_type]=[]
    errbars[samp_type]=[]
    num_samples[samp_type]=[]
    errbar_samps[samp_type] = []
    for pt in possible_pts: #calc errors
        if samp_type in ['randtimes', 'arandtimes', 'symrandtimes']:
            samperrs=[]
            samps = []
            for ave_pt in range(ave_pts):
                app=SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':samp_type, 'numpts':pt})
                util, fmea, phasefmea = resilquant(app, mdl)
                samperrs=samperrs+[calcerror(util_full, util)]
                samps = samps + [len(app.times) + sum([(55-scen.time)/55 for scen in app.scenlist])]
            err=np.mean(samperrs)
            errbar=np.std(samperrs)
            num_samps= np.mean(samps)
            errbar_samp = np.std(samps)
        elif samp_type=='quadrature':
            try:
                quadrature = quadpy.c1.gauss_legendre(pt)
                app=SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'quadrature', 'quad':{'nodes':quadrature.points, 'weights':quadrature.weights}})
                util,  fmea, phasefmea = resilquant(app, mdl)
                err = calcerror(util_full, util)
                errbar=0.0
                num_samps = len(app.times) + sum([(55-scen.time)/55 for scen in app.scenlist])
                errbar_samp = 0.0
            except:
                0
        else:
            app=SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':samp_type, 'numpts':pt})
            util,  fmea, phasefmea = resilquant(app, mdl)
            err = calcerror(util_full, util)
            errbar=0.0
            num_samps = len(app.times) + sum([(55-scen.time)/55 for scen in app.scenlist])
            errbar_samp = 0.0
        errs[samp_type]=errs[samp_type]+[err]
        errbars[samp_type]=errbars[samp_type]+[errbar]
        num_samples[samp_type] = num_samples[samp_type] + [num_samps]
        errbar_samps[samp_type] = errbar_samps[samp_type] + [errbar_samp]


In [42]:
fig=plt.figure(figsize=(6,4))
plt.rcParams.update({'font.size': 11})
errbar_samps['full'] = [0.0]
errs['full']=[0.0]
errbars['full']=[0.0]
num_samples['full'] = [len(app_full.times) + sum([(55-scen.time)/55 for scen in app_full.scenlist])]


app_maxlike = SampleApproach(mdl, faults=[('ExportWater', 'block')], defaultsamp={'samp':'likeliest'})
util_maxlike, fmea_maxlike, phasefmea_maxlike = resilquant(app_maxlike, mdl)
errbar_samps['likeliest'] = [0.0]
errs['likeliest']=[calcerror(util_full, util_maxlike)]
errbars['likeliest']=[0.0]
num_samples['likeliest'] = [len(app_maxlike.times) + sum([(55-scen.time)/55 for scen in app_maxlike.scenlist])]


markers = {"evenspacing":"o", "randtimes":"v", "symrandtimes":"^", "full":"X", "likeliest":"P", "quadrature":"s"}
labels = {"evenspacing":"Evenly-Spaced", "randtimes":"Monte Carlo", "symrandtimes":"Monte Carlo (AV)", "full":"Full", "likeliest":"Likeliest", "quadrature":"Gauss-Legendre"}
#for samp in num_samples:
#    plt.errorbar(num_samples[samp], errs[samp], yerr=errbars[samp],marker = markers[samp], label=samp)
for samp in num_samples:
    if samp in ["evenspacing", "randtimes", "symrandtimes", "quadrature"]:
        if samp=="quadrature":
            p = plt.plot(possible_pts[0:22], errs[samp][0:22],marker = markers[samp], label=labels[samp])
        else:
            p = plt.plot(possible_pts, errs[samp],marker = markers[samp], label=labels[samp])
        if samp in ["symrandtimes", "randtimes"]:
            errline=np.array(errs[samp])
            bars=np.array(errbars[samp])
            plt.fill_between(possible_pts,errline+bars,errline-bars, alpha=0.2,color=p[0].get_color())
    elif samp=="likeliest":
        plt.errorbar(0.3, errs[samp], yerr=errbars[samp],marker = markers[samp], label=labels[samp], markersize=10)
    elif samp=="full":
        plt.errorbar(45, errs[samp], yerr=errbars[samp],marker = markers[samp], label=labels[samp], markersize=10)

#plt.errorbar(range(len(num_samps)), list(num_samps.values(), )
#plt.xticks(range(len(num_samps)), list(num_samps.keys())
plt.legend(loc='upper right',bbox_to_anchor=(0.35, 0., 0.5, 0.48))
plt.xlabel("Number of Points Used")
plt.yscale("symlog", linthreshy=0.001)
plt.ylim([-0.0001,1])
#plt.xscale("log")
plt.grid('both')
plt.ylabel("Error")
plt.title("Error of Sampling Approaches")


In [43]:
plt.savefig("hl_error.pdf", format="pdf", bbox_inches = 'tight', pad_inches = 0)

In [44]:
avgoverallerr = {k: np.mean(v) for k,v in errs.items()}

In [45]:
avgoverallerr